## 텐서보드로 그래프와 학습 곡선 시각화하기

훈련 통계값을 전달하면 브라우저에서 멋진 반응형 그래프를 볼 수 있다.<br/>
계산 그래프의 정의를 사용하여 그래프 구조를 살펴 볼 수 있는 인터페이스를 제공한다. 이는 그래프에서 에러나 병목점 등을 확인하는데 유용하다.

In [46]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import numpy as np

from datetime import datetime

#### 데이터 로드


In [47]:
housing = fetch_california_housing()
rows, cols = housing.data.shape
print(rows, cols)

housing.data[:2]

20640 8


array([[ 8.32520000e+00,  4.10000000e+01,  6.98412698e+00,
         1.02380952e+00,  3.22000000e+02,  2.55555556e+00,
         3.78800000e+01, -1.22230000e+02],
       [ 8.30140000e+00,  2.10000000e+01,  6.23813708e+00,
         9.71880492e-01,  2.40100000e+03,  2.10984183e+00,
         3.78600000e+01, -1.22220000e+02]])

In [48]:
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data[:2]

array([[ 2.34476576,  0.98214266,  0.62855945, -0.15375759, -0.9744286 ,
        -0.04959654,  1.05254828, -1.32783522],
       [ 2.33223796, -0.60701891,  0.32704136, -0.26333577,  0.86143887,
        -0.09251223,  1.04318455, -1.32284391]])

In [49]:
# 훈련 샘플에 대한 편향에 대한 입력 특성 추가
scaled_housing_data_plus_bias = np.c_[np.ones((rows, 1)), scaled_housing_data]
print(scaled_housing_data_plus_bias.shape)
scaled_housing_data_plus_bias[:2]

(20640, 9)


array([[ 1.        ,  2.34476576,  0.98214266,  0.62855945, -0.15375759,
        -0.9744286 , -0.04959654,  1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891,  0.32704136, -0.26333577,
         0.86143887, -0.09251223,  1.04318455, -1.32284391]])

In [63]:
# 학습 설정값
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(rows/batch_size))

In [75]:
# 계산 그래프 정의
X = tf.placeholder(tf.float32, shape=(None, cols+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta = tf.Variable(tf.random_uniform([cols + 1, 1], -1.0, 1.0), name='theta')  #? 
y_pred = tf.matmul(X, theta, name='predictions')

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2 / rows * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

mse_summary = tf.summary.scalar('MES', mse)  # summary라 불리는 텐서보드가 인식하는 이진 로그 문자열에 쓰기 위한 노드를 그래프에 추가
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) # FileWriter 객체를 만들어 로그 디렉터리에 있는 로그 파일에 서머리를 기록한다.

In [76]:
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = "{}/run-{}".format(root_logdir, now)

In [77]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch* n_batches + batch_index)
    indices = np.random.randint(rows, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

In [78]:
init = tf.global_variables_initializer() # 변수 초기화
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE", mse.eval(feed_dict={X: X_batch, y: y_batch}))
            save_path = saver.save(sess, './my_model_{}.ckpt'.format(epoch))
            
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 1 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    save_path = saver.save(sess, './my_model_final.ckpt'.format(epoch))
    file_writer.close()

Epoch 0 MSE 8.819442
Epoch 100 MSE 0.6963695
Epoch 200 MSE 0.60550195
Epoch 300 MSE 0.4282169
Epoch 400 MSE 0.51148313
Epoch 500 MSE 0.47175288
Epoch 600 MSE 0.45941943
Epoch 700 MSE 0.5276242
Epoch 800 MSE 0.59828764
Epoch 900 MSE 0.7635695
